In [4]:
import random
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
import numpy as np
import pydot

from sklearn.externals.six import StringIO
from sklearn.tree import export_graphviz
import pydotplus
import score_solution
from sklearn import preprocessing
from sklearn.metrics import cohen_kappa_score, make_scorer



train_data = pd.read_csv('all_train_data_V2.csv')
test_data = pd.read_csv('all_test_data_V2.csv')

adoptionSpeed = train_data['AdoptionSpeed'] 
del train_data['AdoptionSpeed']



In [5]:
test_ids = test_data[['PetID']]

del train_data["Name"]
del train_data["RescuerID"]
del train_data["Description"]
del train_data["PetID"]


del test_data["Name"]
del test_data["RescuerID"]
del test_data["Description"]
del test_data["PetID"]

train_data = train_data.fillna(0)
test_data = test_data.fillna(0)
train_data = preprocessing.scale(train_data)
test_data = preprocessing.scale(test_data)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.


In [7]:
from sklearn.datasets import load_boston
import xgboost as xgb
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier



gbc_model = GradientBoostingClassifier(n_estimators=1100, learning_rate=0.03,
     max_depth=2)
agb_model = AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=0.03, n_estimators=1100, random_state=None)
extra_model = ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

rf_model = RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
bagging_model = BaggingClassifier(DecisionTreeClassifier(), n_estimators = 1000,
                            max_samples=0.5, max_features=0.5)


xgb_model = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5, gamma=0, learning_rate=0.03, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=650,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

classifiers = [
    ('gbc', gbc_model), ('agb', agb_model),
    ('extra', extra_model), ('rf', rf_model),
    ('bagging', bagging_model), ("xgb", xgb_model)
]

voting_model = VotingClassifier(estimators=classifiers, voting='hard')

voting_model.fit(train_data, adoptionSpeed)

ans = voting_model.predict(test_data)

ans


array([1, 4, 4, ..., 2, 1, 3])

In [8]:
submission = pd.DataFrame({'PetID': test_ids['PetID'].values, 'AdoptionSpeed': ans})
print(submission.head())
submission.to_csv('submission2.csv', index=False)



       PetID  AdoptionSpeed
0  000c21f80              1
1  001d503e8              4
2  001ec8d5b              4
3  002d58f95              2
4  004682b50              4


In [11]:
voting_model.estimators_

[GradientBoostingClassifier(criterion='friedman_mse', init=None,
               learning_rate=0.03, loss='deviance', max_depth=2,
               max_features=None, max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=1100,
               n_iter_no_change=None, presort='auto', random_state=None,
               subsample=1.0, tol=0.0001, validation_fraction=0.1,
               verbose=0, warm_start=False),
 AdaBoostClassifier(algorithm='SAMME.R',
           base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
             max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=None,
             splitter='best'),
           learning_r